<a href="https://colab.research.google.com/github/Premmikan/Comsys/blob/main/Premmika_NIEMPREM_ch03_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sequential Logic and Memory Devices
*ดัดแปลงจากโน้ตบุ๊คต้นฉบับโดย ผศ.ดร.จิตร์ทัศน์ ฝักเจริญผล ภาคต้น ปีการศึกษา 2563 และอ้างอิงเนื้อหาจากตำรา [The Elements of Computing Systems โดย Nisan และ Schocken](https://www.nand2tetris.org/book)*

**เอกสารประกอบ:** อ่านเอกสารประกอบจากตำรา [รากฐานระบบคอมพิวเตอร์: จากเม็ดทรายสู่ Hello, World! บทที่ 3](https://ecourse.cpe.ku.ac.th/courses/comsys/comsys-book.pdf#page=76)

บทนี้เราจะศึกษาวงจรแบบ sequential โดยใช้เกท DFF เป็นเกทพื้นฐานร่วมกับอุปกรณ์พื้นฐานต่าง ๆ ที่เราสร้างมาเมื่อบทที่แล้ว

ขั้นแรกเราจะโหลดไลบรารี

In [ ]:
%%capture
!rm -rf comp-sys-public-lib
!wget -q -O - https://ecourse.cpe.ku.ac.th/courses/comsys/lib/ch03.tgz | tar zxf -
!pip install -e comp-sys-public-lib
!pip install -e comp-sys-public-lib/modules/component-builder
import site; site.main()
from course_ch03_init import *

ภายในไลบรารีจะมีเกทพื้นฐานที่สร้างขึ้นมาจากแบบฝึกหัดก่อนให้พร้อมใช้งานได้ทันทีตามรายการ นิสิตไม่จำเป็นต้องสร้างเกทเหล่านี้ขึ้นมาใหม่

* [`Nand`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/nand.html),
[`And`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/and.html),
[`Or`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/or.html),
[`Not`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/not.html),
[`Buffer`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/buffer.html) (สำหรับเชื่อมต่อ wire),
[`Mux`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/mux.html),
[`DMux`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/dmux.html)
* [`Not16`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/not16.html),
[`And16`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/and16.html),
[`Or16`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/or16.html),
[`Mux16`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/mux16.html),
[`Or8Way`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/or8way.html),
[`Mux4Way16`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/mux4way16.html),
[`Mux8Way16`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/mux8way16.html),
[`DMux4Way`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/dmux4way.html),
[`DMux8Way`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/dmux8way.html)
* [`HalfAdder`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/halfadder.html),
[`FullAdder`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/fulladder.html),
[`Add16`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/add16.html),
[`Inc16`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/inc16.html),
[`ALU`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/alu.html)

# แนะนำอุปกรณ์​ DFF

หน่วยย่อยพื้นฐานสำหรับอุปกรณ์หน่วยความจำที่เราจะใช้คือ `DFF` (Data Flip Flop หรือ Delay Flip Flop) ซึ่งมีสัญลักษณ์ดังภาพ

![dff.png](https://ecourse.cpe.ku.ac.th/courses/comsys/pics/dff.png)


`DFF` จะทำงานโดยอ้างอิงกับสัญญาณนาฬิกา `Clk` โดยลอจิกที่ขาเอาท์พุท `Q` จะถูกกำหนดให้มีค่าเท่ากับลอจิกที่ขาอินพุท `D` ณ จังหวะที่สัญญาณ `Clk` เปลี่ยนจาก 0 เป็น 1 เท่านั้น (rising edge trigger) และจะคงสภาพอยู่เช่นนั้นโดยไม่สนใจการเปลี่ยนแปลงลอจิกของขา `D` จนกว่า `Clk` จะเปลี่ยนจาก 0 เป็น 1 อีกครั้ง

DFF สามารถสร้างขึ้นจากวงจรลอจิกพื้นฐานล้วน ๆ ได้ดังภาพ (ภาพจาก Wikipedia)

![dff.svg](https://upload.wikimedia.org/wikipedia/commons/3/37/D-Type_Flip-flop_Diagram.svg)

ทดลองศึกษาพฤติกรรมของวงจรข้างต้นผ่าน [Circuit Simulator](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKDAQmwRXDB4BZ+eXjyo8kCNhxBceKBIRCDh8xWKgUpEDjz7CExXSmHraWinxAp+tQ1ZsbxmgJJX8MtFbzDsnqjAkWV2xCH09lD38oaED2CAxIfnBBEAw8JLAU9VjpBKTjYTT870dqSTiQPEIMwipiFLBa0rMKqvyMHnr20SsylgB3LzCqXCTffwGZRPBjGW4ZkxYAGTnOxTAMRWI1DQAzAEMAGwBnOmpISdkrDqnuqEvpzB5RyPvBl6fbhbelIRFf4SNCaDCJAmQhKzWH7YCEoKEwxTWYHgxEOK5In4RVQWOQKTF-ZjiSyEn4GIzCWqA2YXQaU755b40kB0vTM9xgpkspr8bZuRYg3loYQ8xFQpkiyFJHSS0nElISzJJcW8xVKFVNZXrJp0oU-HXuHUOTnuDFk+zIl7jGShV7iv5Wwg3K2cp2eR3PaYuj1JBkvL1fd3gDWTQNghkcyZFZJJPBwaM-WNUVWJ8CbBNxjaKBkYpnZhzZm659JDSrVEtMlPJsuqpldIN1JoRwbbJNNFuptSTduZ0v5I2TNrm3tWPG1lKfdufCvVuyDnu1poTxejrvLxRzuwXACyIG7jcSraiKBiLB3k5uYAP1x6x8kK11Vp7zr2R1O5xYQA)

ในทางปฏิบัติ วงจร `DFF` อาจสร้างขึ้นด้วยเทคนิคอื่นแทนการสร้างจากวงจรลอจิกข้างต้นโดยตรง เช่น [Pass Transistor Logic](https://en.wikipedia.org/wiki/Pass_transistor_logic) ซึ่งช่วยลดจำนวนทรานซิสเตอร์ภายในวงจรลงได้

อย่างไรก็ตามในรายวิชานี้เราจะไม่สร้าง `DFF` ขึ้นมาจากเกทพื้นฐาน แต่จะถือว่าเป็นอุปกรณ์พื้นฐานที่พร้อมนำมาใช้งานได้ทันที และเพื่อให้สอดคล้องกับตำราเรียน เราจะตั้งชื่อขาอินพุทว่า `In` ขาเอาท์พุทว่า `out` และขาสัญญาณนาฬิกาว่า `clk`

ลองศึกษาพฤติกรรมของ `DFF` ในโหมดโต้ตอบด้านล่าง

In [ ]:
# run to see interactive mode for DFF
interact(DFF)

เนื่องจาก `DFF` จะทำงานตามขา `clk` ดังนั้นเมื่อนำ `DFF` ไปสร้างเป็นอุปกรณ์อื่น ๆ อุปกรณ์นั้นก็จะต้องทำงานตาม `clk` ไปด้วยเช่นกัน เราเรียกอุปกรณ์เหล่านี้ว่า *clocked components* และถือว่าอุปกรณ์ทุกตัวมีขา `clk` ที่เชื่อมถึงกันทั้งหมด

ดังนั้นเพื่อความสะดวกในการจำลองและการใช้ไลบรารี นิสิตสามารถพิจารณาได้ว่าสัญญาณนาฬิกาจะถูก **ผูกเข้ากับ clocked component โดยอัตโนมัติ** และจะมองไม่เห็นตอนที่เขียนนิยามคลาสของอุปกรณ์ แต่จะปรากฏขึ้นเองเมื่อเรียกใช้งานฟังก์ชัน `interact`

นอกจากนี้ การเรียกฟังก์ชัน `eval` กับอุปกรณ์หนึ่งครั้งจะถือว่าเป็นการทริกเกอร์สัญญาณนาฬิกาไปพร้อมกันหนึ่งครั้งเช่นกัน (โดยไม่ต้องป้อนสัญญาณนาฬิกาขึ้นและลงด้วยตนเอง)

ลองพิจารณาอุปกรณ์ด้านล่างต่อไปนี้เพื่อความเข้าใจที่มากขึ้น

In [ ]:
class DFFDemo(Component):
    IN = [w.In]
    OUT = [w.out1, w.out2, w.out3]

    PARTS = [
        Buffer(In=w.In, out=w.out1),
        DFF(In=w.In, out=w.out2),
        DFF(In=w.out2, out=w.out3),
    ]

ทดลองเรียก `interact`

In [ ]:
interact(DFFDemo, depth=1)

ทดลองติดตามสัญญาณด้วย​ `trace` กรณีนี้ไลบรารีจะจัดการกำเนิดสัญญาณนาฬิการให้อัตโนมัติทุก ๆ ช่วงบิตของอินพุท จึงไม่จำเป็นต้องระบุค่าอินพุทให้กับขา `clk`

In [ ]:
dffdemo1 = DFFDemo()
trace(dffdemo1, {'In':'010100110011000111000111'}, probes=['out1', 'out2', 'out3'])

{'out1': '010100110011000111000111',
 'out2': '001010011001100011100011',
 'out3': '000101001100110001110001'}

ทดลองพล็อตผลการติดตามสัญญาณ

In [ ]:
dffdemo2 = DFFDemo()
trace_and_plot_inout(dffdemo2, {'In':'010100110011000111000111'})

## Counter สองบิต

เราสามารถใช้ `DFF` กับ `FullAdder` ทำ counter ขนาด 2 บิตได้ดังด้านล่าง

**หมายเหตุ:** เราจงใจแยกสัญญาณ `w.updated_counter` เป็นสองบิตผ่าน `Buffer` เพื่อให้ทดลองเห็นใน `interact` ได้ชัดเจนยิ่งขึ้น

In [ ]:
class Inc2(Component):
    IN = [w(2).In]
    OUT = [w(2).out]

    PARTS = [
        FullAdder(a=w.In[0], b=w.T, c=w.F, sum=w.out[0], carry=w.carry0),
        FullAdder(a=w.In[1], b=w.F, c=w.carry0, sum=w.out[1], carry=w.carry1),
    ]

class Counter2(Component):
    IN = []
    OUT = [w(2).out]

    PARTS = [
        DFF(In=w.updated_counter0, out=w.out[0]),
        DFF(In=w.updated_counter1, out=w.out[1]),
        Inc2(In=w.out, out=w.updated_counter),
        Buffer(In=w(2).updated_counter[0], out=w.updated_counter0),
        Buffer(In=w.updated_counter[1], out=w.updated_counter1),
    ]

ทดลอง `interact`

In [ ]:
interact(Counter2, depth=1)

ทดลอง `trace`

In [ ]:
c = Counter2()
trace_and_plot_inout(c, {}, step=20)

# `Bit` และ `Register`

รีจิสเตอร์ (Register) คือหน่วยความจำที่รองรับการกำหนดค่าและเรียกคืนค่ากลับออกมาได้ หลัก ๆ จะมีขานำเข้า `In` และ `load` ถ้า `load` เป็นจริง ผลลัพธ์ `out` ในช่วงสัญญาณนาฬิกาลูกถัดไปจะเท่ากับค่า `In` ในเวลาปัจจุบัน

เราจะเริ่มด้วยการสร้างอุปกรณ์ `Bit` ที่เป็นรีจีสเตอร์ขนาด 1 บิต และอุปกรณ์ `Register` ที่เป็นรีจีสเตอร์ขนาด 16 บิต

## `Bit`

`Bit` คือ register ขนาด 1 บิต  ถ้าจะเขียนให้ชัดเจนคือ ผลลัพธ์ `out` ของ `Bit` จะมีคุณสมบัติดังนี้

* ถ้า `load(t-1)` เป็นจริงแล้ว
 * `out(t)` จะเท่ากับ `In(t-1)`
* ไม่เช่นนั้น
 * `out(t)` จะเท่ากับ `out(t-1)`

ในการออกแบบ `Bit` เราจะต้องเลือกว่าจะให้ค่า `out` ในสัญญาณนาฬิกาลูกถัดไปมีค่าเป็นเท่าใด โดยใช้ค่าจากขา `load` เป็นตัวเลือกป้อนลอจิกให้กับ `DFF` (คำใบ้: ใช้ `Mux`)

ตัวอย่างการทำงานในโหมดโต้ตอบ: [`Bit`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/bit.html)

In [ ]:
# DO NOT ERASE THIS CELL - to be graded
class Bit(Component):
    IN = [w.In, w.load]
    OUT = [w.out]

    PARTS = [
        Mux(a = w.out, b = w.In, sel = w.load, out = w.x),
        DFF(In = w.x, out = w.out)
    ]

In [ ]:
# พื้นที่ทดสอบ (ปรับเปลี่ยนตามต้องการ)
interact(Bit, depth=1)

### Test case

In [ ]:
class TestBit(unittest.TestCase):
    def setUp(self):
        self.bit = Bit()

    def test_inputTrace(self):
        self.assertEqual(
            trace(
                self.bit,
                {'In':'11001101101011001', 'load':'10010000110101010'},
                ['out']
            ),
            {'out': '01110000010000110'}
        )

run_test(TestBit)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


## `Register`

`Register` คือการเอา `Bit` มาต่อกัน 16 ตัว รับค่าและส่งออกค่าทีละ 16 บิต

ตัวอย่างการทำงานในโหมดโต้ตอบ: [`Register`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/register.html)

In [ ]:
from json import load
# DO NOT ERASE THIS CELL - to be graded
class Register(Component):
    IN = [w(16).In, w.load]
    OUT = [w(16).out]

    PARTS = [
    ]
    for i in range(16):
      PARTS.append(Bit(In = w.In[i], load = w.load, out = w.out[i]))


In [ ]:
# พื้นที่ทดสอบ (ปรับเปลี่ยนตามต้องการ)
trace_and_plot_inout(Register(), {
    'In':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],
    'load':'01011111011101010',
})
interact(Register)

### Test case

In [ ]:
class TestRegister(unittest.TestCase):
    def setUp(self):
        self.register = Register()

    def test_inputTrace(self):
        self.assertEqual(
            trace(
                self.register,
                {'In':[1,2,3,0,1,2,65535,32769,0,0,10,10,65534,65534,1,15,0], 'load':'01011111011101010'},
                ['out']
            ),
            {'out': [0,0,2,2,0,1,2,65535,32769,32769,0,10,10,10,65534,65534,15]}
        )

run_test(TestRegister)

.
----------------------------------------------------------------------
Ran 1 test in 0.042s

OK


# แรม (RAM)

ในส่วนนี้เราจะสร้างหน่วยความจำแรมที่มีขนาดต่าง ๆ โดยทยอยสร้างจากขนาด 8 เวิร์ด (1 เวิร์ดเท่ากับ 16 บิต) 64 เวิร์ด ไปจนถึง 16K เวิร์ด

แรมทุกขนาดมีพฤติกรรมเช่นเดียวกันหมด แตกต่างกันที่ความเป็นไปได้ของค่า `address`
* `out(t)` = `RAM[address(t)](t)`
* ถ้า `load(t-1)` แล้ว `RAM[address(t-1)](t) = in(t-1)`

## RAM8

`RAM8` เป็นแรมที่เก็บข้อมูลได้ 8 เวิร์ด ระบุแอดเดรส 3 บิต (0 ถึง 7) สร้างขึ้นจาก `Register` 8 ตัว

![ram8.png](https://ecourse.cpe.ku.ac.th/courses/comsys/pics/ram8.png)

แนวทางการออกแบบ
* ส่วนของเอาท์พุท ใช้อุปกรณ์ที่เลือกได้ว่าจะเอาเอาท์พุทของรีจีสเตอร์ใดมาเป็นผลลัพธ์
* ส่วนของอินพุท ใช้อุปกรณ์ที่ควบคุมได้ว่าจะเลือกส่งลอจิก 1 ไปให้ขา `load` ของรีจีสเตอร์ใดเพื่อให้รับค่าอินพุทไปบันทึกไว้
* อุปกรณ์ที่กล่าวมาทั้งสองตัวได้ทำเอาไว้แล้วตั้งแต่คาบก่อน ๆ

ตัวอย่างการทำงานในโหมดโต้ตอบ: [`RAM8`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/ram8.html)

In [ ]:
# DO NOT ERASE THIS CELL - to be graded
class RAM8(Component):
    IN = [w(16).In, w(3).address, w.load]
    OUT = [w(16).out]

    PARTS = [
        DMux8Way(In = w.load , sel = w.address[0:3], a = w.load0, b = w.load1, c = w.load2, d = w.load3, e = w.load4, f = w.load5, g = w.load6, h = w.load7),
    ]
    for i in range(8):
      PARTS.append(Register(In = w.In, load = getattr(w,f"load{i}"), out = getattr(w,f"out{i}")))
    PARTS.append(Mux8Way16(a = w(16).out0, b = w(16).out1, c = w(16).out2, d = w(16).out3, e = w(16).out4, f = w(16).out5, g = w(16).out6, h = w(16).out7, sel = w.address[0:3], out = w.out))

In [ ]:
# พื้นที่ทดสอบ (ปรับเปลี่ยนตามต้องการ)
interact(RAM8, depth=1)

### Test case

In [ ]:
class TestRAM8(unittest.TestCase):
    def setUp(self):
        self.ram8 = RAM8()

    def is_fast_ram(self, ram):
        return (getattr(ram, 'is_fast_ram', False)
            or (len(ram.PARTS) == 1))

    def test_not_fast_ram(self):
        self.assertEqual(self.is_fast_ram(self.ram8), False,
                        'Your current RAM component is a FastRAM, '
                        'please re-evaluate the cell where you define your '
                        'own RAM implementation')

    def test_inputTrace(self):
        self.assertEqual(
            trace(
                self.ram8,
                {'In':[2,2,10,10,5,5,5,12,9,13,15,0,0,16,13,7,7,7,9,9,9,255,0,0,0,0,8,8,7],
                 'address':[0,0,0,1,1,0,7,7,7,7,4,4,1,5,5,5,5,1,1,1,2,2,2,2,4,1,3,3,3],
                 'load':'01010011101001100100110000010'
                },
                ['out']
            ),
            {'out': [0,0,2,0,10,2,0,5,12,9,0,15,10,0,16,13,13,10,7,7,0,9,255,255,15,7,0,0,8]}
        )


run_test(TestRAM8)

..
----------------------------------------------------------------------
Ran 2 tests in 2.211s

OK


## RAM64

ในส่วนนี้เราจะสร้าง `RAM64` จาก `RAM8` ทั้งหมด 8 ตัว

![ram64.png](https://ecourse.cpe.ku.ac.th/courses/comsys/pics/ram64.png)

`RAM8` ในตอนนี้ประกอบไปด้วยเกทจำนวนมาก หากเรานำมาประกอบเป็นแรมที่ใหญ่ขึ้นจะทำให้การจำลองการทำงานช้าลงไปเรื่อย ๆ เนื่องจากการทำงานทุกอย่างถูกจำลองขึ้นด้วยซอฟต์แวร์ เพื่อเพิ่มความเร็วในการจำลองวงจร เราจะเปลี่ยนคลาส `RAM8` ที่จะเอามาใช้ใน `RAM64` ให้เป็นคลาสที่มีประสิทธิภาพมากกว่า เราได้วางคำสั่งเพื่อสร้างคลาสดังกล่าวไว้ตอนต้นของนิยาม `RAM64` แล้ว  อย่างไรก็ตาม ถ้านิสิตสร้างคลาส `RAM8` นี้ขึ้นมาทับ `RAM8` ของเดิมไปแล้ว คลาสนี้จะไม่ผ่าน test case ของ `RAM8` อีกต่อไป  ถ้าต้องการกลับไปรันตัวทดสอบ จะต้องไปรันเซลล์ที่นิยาม `RAM8` เดิมไว้เองอีกทีหนึ่ง

ตัวอย่างการทำงานแบบโต้ตอบ: [`RAM64`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/ram64.html)

In [ ]:
# DO NOT ERASE THIS CELL - to be graded

# replace RAM8 with its fast implementation
RAM8 = gen_fast_ram_component(3)

class RAM64(Component):
    IN = [w(16).In, w(6).address, w.load]
    OUT = [w(16).out]

    PARTS = [
        DMux8Way(In = w.load , sel = w.address[3:6], a = w.load0, b = w.load1, c = w.load2, d = w.load3, e = w.load4, f = w.load5, g = w.load6, h = w.load7),
    ]
    for i in range(8):
      PARTS.append(RAM8(In = w.In, address = w.address[0:3], load = getattr(w,f"load{i}"), out = getattr(w,f"out{i}")))
    PARTS.append(Mux8Way16(a = w(16).out0, b = w(16).out1, c = w(16).out2, d = w(16).out3, e = w(16).out4, f = w(16).out5, g = w(16).out6, h = w(16).out7, sel = w.address[3:6], out = w.out))

In [ ]:
# พื้นที่ทดสอบ (ปรับเปลี่ยนตามต้องการ)
interact(RAM64, depth=1)

### Test case

In [ ]:
class TestRAM64(unittest.TestCase):
    def setUp(self):
        self.ram64_1 = RAM64()
        self.ram64_2 = RAM64()

    def is_fast_ram(self, ram):
        return (getattr(ram, 'is_fast_ram', False)
            or (len(ram.PARTS) == 1))

    def test_not_fast_ram(self):
        self.assertEqual(self.is_fast_ram(self.ram64_1), False,
                         'Your current RAM component is a FastRAM, '
                         'please re-evaluate the cell where you define your '
                         'own RAM implementation')

    def test_1(self):
        self.assertEqual(
            trace(
                self.ram64_1,
                {'In':[2,2,10,10,5,5,5,12,9,13,15,0,0,16,13,7,7,7,9,9,9,255,0,0,0,0,8,8,7],
                 'address':[0,0,0,1,1,0,57,57,57,57,14,14,1,35,35,35,35,1,1,1,2,2,2,2,14,1,63,63,63],
                 'load':'01010011101001100100110000010'
                },
                ['out']
            ),
            {'out': [0,0,2,0,10,2,0,5,12,9,0,15,10,0,16,13,13,10,7,7,0,9,255,255,15,7,0,0,8]}
        )

    def test_2(self):
        self.assertEqual(
            trace(
                self.ram64_2,
                {'In':[0,0,10,10, 10,3,3,3, 7,7,7,7, 32769,32769,32769,32769, 32769,32769,32769,32769, 32769,15,15,32769, 15,15,32769,15, 15,32769,15,15, 32769,15,15,32769, 15,15,32769,15, 15,32769,15,15, 32769,32769,8],
                 'address':[0,0,0,1, 0,3,3,3, 1,7,7,0, 0,1,2,3, 4,5,6,7, 0,0,0,0, 1,0,1,2, 0,2,3,0, 3,4,0,4, 50,0,50,60, 0,60,57,0, 57,0,57],
                 'load':'01010010010011111111010110110110110110110110100'
                },
                ['out']
            ),
            {'out': [0,0,0,0, 0,0,0,3, 10,0,7,0, 0,10,0,3, 0,0,0,7, 32769,32769,15,15, 32769,32769,15,32769, 32769,15,32769,32769, 15,32769,32769,15, 0,32769,15,0, 32769,15,0,32769, 15,32769,32769]}
        )

class TestRAMBase(unittest.TestCase):
    def perform_test_and_check(self, ram, In, load, address):
        inputs = {'In': In,
                  'load': load,
                  'address': address,}

        sim_ram = [0] * 100000
        expected_out = []
        for r in range(len(In)):
            expected_out.append(Signal(sim_ram[address[r]], 16))
            if load[r] == 1:
                sim_ram[address[r]] = In[r]

        tr = trace(ram, inputs, ['out'])
        for r in range(len(In)):
            self.assertEqual(tr['out'][r].value, expected_out[r].value)

    def do_test_random(self, ram, length, max_address=63):
        from random import randint, choice

        rand_addresses = [randint(0,max_address) for i in range(max([length, 50]))]

        In = []
        load = []
        address = []
        for l in range(length):
            In.append(randint(0,65535))
            load.append(randint(0,1))
            address.append(choice(rand_addresses))

        self.perform_test_and_check(ram, In, load, address)

    def do_test(self, ram):
        In = [0,0,10,10,10,3,3,3,7,7,7,7,32769,32769,32769,32769,32769,32769,32769,32769,32769,15,15,32769,15,15,32769,15,15,32769,15,15,32769,15,15,32769,15,15,32769,15,15,32769,15,15,32769,32769]
        load = [0,1,0,1,0,0,1,0,0,1,0,0,1,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,0,1,1,0,1,1,0,1,1,0,1,1,0,1,1,0,1,0]
        address = [0,0,0,1,0,3,3,3,1,7,7,0,0,1,2,3,4,5,6,7,0,0,0,0,1,0,1,2,0,2,3,0,3,4,0,4,50,0,50,60,0,60,57,0,57,0]

        self.perform_test_and_check(ram, In, load, address)

class TestRAM64Automated(TestRAMBase):
    def test_ram(self):
        ram64 = RAM64()
        self.do_test_random(ram64, 500)


run_test(TestRAM64)
run_test(TestRAM64Automated)


...
----------------------------------------------------------------------
Ran 3 tests in 1.870s

OK
.
----------------------------------------------------------------------
Ran 1 test in 6.222s

OK


## RAM512

`RAM512` สร้างขึ้นจาก `RAM64` 8 ตัว เช่นเดียวกับตอนสร้าง `RAM64` เราจะเปลี่ยนคลาส `RAM64` ที่นำไปใช้ใน `RAM512` ให้เป็นตัวที่เร็วขึ้นเพื่อเพิ่มประสิทธิภาพการจำลองการทำงาน

ตัวอย่างการทำงานแบบโต้ตอบ: [`RAM512`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/ram512.html)

In [ ]:
# DO NOT ERASE THIS CELL - to be graded

# replace RAM64 with its fast implementation
RAM64 = gen_fast_ram_component(6)

class RAM512(Component):
    IN = [w(16).In, w(9).address, w.load]
    OUT = [w(16).out]

    PARTS = [
        DMux8Way(In = w.load , sel = w.address[6:9], a = w.load0, b = w.load1, c = w.load2, d = w.load3, e = w.load4, f = w.load5, g = w.load6, h = w.load7),
    ]
    for i in range(8):
      PARTS.append(RAM64(In = w.In, address = w.address[0:6], load = getattr(w,f"load{i}"), out = getattr(w,f"out{i}")))
    PARTS.append(Mux8Way16(a = w(16).out0, b = w(16).out1, c = w(16).out2, d = w(16).out3, e = w(16).out4, f = w(16).out5, g = w(16).out6, h = w(16).out7, sel = w.address[6:9], out = w.out))

In [ ]:
# พื้นที่ทดสอบ (ปรับเปลี่ยนตามต้องการ)
interact(RAM512, depth=1)

### Test case

In [ ]:
class TestRAM512(unittest.TestCase):
    def setUp(self):
        self.ram512 = RAM512()

    def is_fast_ram(self, ram):
        return (getattr(ram, 'is_fast_ram', False)
            or (len(ram.PARTS) == 1))

    def test_not_fast_ram(self):
        self.assertEqual(self.is_fast_ram(self.ram512), False,
                         'Your current RAM component is a FastRAM, '
                         'please re-evaluate the cell where you define your '
                         'own RAM implementation')

    def test_1(self):
        self.assertEqual(
            trace(
                self.ram512,
                {'In':[0,0,10,10, 10,3,3,3, 7,7,7,7, 32769,32769,32769,32769, 32769,32769,32769,32769, 32769,15,15,32769, 15,15,32769,15, 15,32769,15,15, 32769,15,15,32769, 15,15,32769,15, 15,32769,15,15, 32769,32769,8],
                 'address':[0,0,0,111, 0,333,333,333, 111,77,77,0, 0,111,222,333, 444,5,6,77, 0,0,0,0, 111,0,111,222, 0,222,333,0, 333,444,0,444, 500,0,500,60, 0,60,511,0, 511,0,511],
                 'load':'01010010010011111111010110110110110110110110100'
                },
                ['out']
            ),
            {'out': [0,0,0,0, 0,0,0,3, 10,0,7,0, 0,10,0,3, 0,0,0,7, 32769,32769,15,15, 32769,32769,15,32769, 32769,15,32769,32769, 15,32769,32769,15, 0,32769,15,0, 32769,15,0,32769, 15,32769,32769]}
        )

class TestRAM512Random(TestRAMBase):
    def test_random(self):
        ram512 = RAM512()
        self.do_test_random(ram512, 500, 511)

run_test(TestRAM512)
run_test(TestRAM512Random)

..
----------------------------------------------------------------------
Ran 2 tests in 1.093s

OK
.
----------------------------------------------------------------------
Ran 1 test in 6.699s

OK


## RAM4K

`RAM4K` สร้างขึ้นจาก `RAM512` 8 ตัว เช่นเดียวกับตอนสร้าง `RAM512` เราจะเปลี่ยนคลาส `RAM512` ที่นำไปใช้ใน `RAM4K` ให้เป็นตัวที่เร็วขึ้นเพื่อเพิ่มประสิทธิภาพการจำลองการทำงาน

ตัวอย่างการทำงานแบบโต้ตอบ: [`RAM4K`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/ram4k.html)

In [ ]:
# DO NOT ERASE THIS CELL - to be graded

# replace with fast implementation
RAM512 = gen_fast_ram_component(9)

class RAM4K(Component):
    IN = [w(16).In, w(12).address, w.load]
    OUT = [w(16).out]

    PARTS = [
        DMux8Way(In = w.load , sel = w.address[9:12], a = w.load0, b = w.load1, c = w.load2, d = w.load3, e = w.load4, f = w.load5, g = w.load6, h = w.load7),
    ]
    for i in range(8):
      PARTS.append(RAM512(In = w.In, address = w.address[0:9], load = getattr(w,f"load{i}"), out = getattr(w,f"out{i}")))
    PARTS.append(Mux8Way16(a = w(16).out0, b = w(16).out1, c = w(16).out2, d = w(16).out3, e = w(16).out4, f = w(16).out5, g = w(16).out6, h = w(16).out7, sel = w.address[9:12], out = w.out))

In [ ]:
# พื้นที่ทดสอบ (ปรับเปลี่ยนตามต้องการ)
interact(RAM4K, depth=1)

### Test case

In [ ]:
class TestRAM4KRandom(TestRAMBase):

    def is_fast_ram(self, ram):
        return (getattr(ram, 'is_fast_ram', False)
            or (len(ram.PARTS) == 1))

    def test_not_fast_ram(self):
        ram = RAM4K()
        self.assertEqual(self.is_fast_ram(ram), False,
                         'Your current RAM component is a FastRAM, '
                         'please re-evaluate the cell where you define your '
                         'own RAM implementation')

    def test_random(self):
        ram4K = RAM4K()
        self.do_test_random(ram4K, 500, 4095)

run_test(TestRAM4KRandom)

..
----------------------------------------------------------------------
Ran 2 tests in 6.521s

OK


## RAM16K

`RAM16K` เป็นผลลัพธ์สุดท้ายของการสร้างระบบหน่วยความจำ ซึ่งสร้างขึ้นจาก `RAM4K` 4 ตัว เช่นเดียวกับตอนสร้าง `RAM4K` เราจะเปลี่ยนคลาส `RAM4K` ที่นำไปใช้ใน `RAM16K` ให้เป็นตัวที่เร็วขึ้นเพื่อเพิ่มประสิทธิภาพการจำลองการทำงาน

ตัวอย่างการทำงานแบบโต้ตอบ: [`RAM16K`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/ram16k.html)

In [ ]:
# DO NOT ERASE THIS CELL - to be graded

# replace with fast implementation
RAM4K = gen_fast_ram_component(12)

class RAM16K(Component):
    IN = [w(16).In, w(14).address, w.load]
    OUT = [w(16).out]

    PARTS = [
        DMux4Way(In = w.load , sel = w.address[12:14], a = w.load0, b = w.load1, c = w.load2, d = w.load3, e = w.load4, f = w.load5, g = w.load6, h = w.load7),
    ]
    for i in range(4):
      PARTS.append(RAM4K(In = w.In, address = w.address[0:12], load = getattr(w,f"load{i}"), out = getattr(w,f"out{i}")))
    PARTS.append(Mux4Way16(a = w(16).out0, b = w(16).out1, c = w(16).out2, d = w(16).out3, e = w(16).out4, f = w(16).out5, g = w(16).out6, h = w(16).out7, sel = w.address[12:14], out = w.out))

In [ ]:
# พื้นที่ทดสอบ (ปรับเปลี่ยนตามต้องการ)
interact(RAM16K, depth=1)

### Test case

In [ ]:
class TestRAM16KRandom(TestRAMBase):

    def is_fast_ram(self, ram):
        return (getattr(ram, 'is_fast_ram', False)
            or (len(ram.PARTS) == 1))

    def test_not_fast_ram(self):
        ram = RAM16K()
        self.assertEqual(self.is_fast_ram(ram), False,
                         'Your current RAM component is a FastRAM, '
                         'please re-evaluate the cell where you define your '
                         'own RAM implementation')

    def test_random(self):
        ram16k = RAM16K()
        self.do_test_random(ram16k, 500, 16000)

run_test(TestRAM16KRandom)

..
----------------------------------------------------------------------
Ran 2 tests in 2.329s

OK


# Program Counter (`PC`)

ในส่วนสุดท้ายเราจะสร้าง Program Counter (`PC`) ซึ่งจะเป็นรีจีสเตอร์แบบพิเศษที่ใช้เก็บและจัดการแอดเดรสของคำสั่งปัจจุบัน รีจีสเตอร์นี้นอกจากจะเก็บค่าใหม่ได้ด้วยขา `load` แล้ว ยังมีฟีเจอร์เพิ่มเติมดังนี้

* สามารถรีเซ็ตค่าให้เป็น 0 ได้ด้วยขา `reset`
* สามารถสั่งให้เพิ่มค่าอัตโนมัติทุก ๆ รอบสัญญาณนาฬิกาด้วยขา `inc` (เทียบได้กับการเลื่อนแอดเดรสไปยังคำสั่งถัดไป)  

ในบรรดาขาควบคุมทั้ง 3 ขานี้ ขา `reset` จะมีลำดับความสำคัญสูงสุด ตามด้วย `load` และตามด้วย `inc`

ตัวอย่างการทำงานแบบโต้ตอบ: [`PC`](https://ecourse.cpe.ku.ac.th/courses/comsys/demo/pc.html)

ในการสร้าง `PC` นิสิตอาจต้องสร้างอุปกรณ์เสริมอื่น ๆ ขึ้นมาใช้งานเพิ่มเติม ให้นิยามอุปกรณ์ไว้ในเซลล์เดียวกันกับ `PC` ด้านล่างนี้

In [ ]:
# DO NOT ERASE THIS CELL - to be graded

# put additional auxilary components here (if needed)

class PC(Component):
    IN = [w(16).In, w.load, w.inc, w.reset]
    OUT = [w(16).out]

    PARTS = [
        Inc16(In = w.out, out = w(16).out0),
        Mux16(a = w.out,b = w.out0, sel = w.inc,out = w(16).out1),
        Mux16(a = w.out1, b = w.In, sel = w.load, out = w(16).out2),
        Mux16(a = w.out2,b = w(16).F, sel = w.reset,out = w(16).out3),
        Register(In = w.out3, load = w.T , out = w.out)
    ]

In [ ]:
# พื้นที่ทดสอบ (ปรับเปลี่ยนตามต้องการ)
trace_and_plot_inout(PC(), {
        'In'   :[5,5,5,5,8,11,14,14,14,63,63,63,63,128,15,15,15,15,15,15,15,15,143,143,143,10,10,10,10,10,13,15,15],
        'load' :'000111000000110000000110000011100',
        'inc'  :'011110001111111111111000011111100',
        'reset':'000000000000000001000000000010000'
    },
    ['out']
)
interact(PC, depth=1)

### Test case

In [ ]:
class TestPC(unittest.TestCase):
    def setUp(self):
        self.pc = PC()

    def test_inputTrace(self):
        self.assertEqual(
            trace(
                self.pc,
                {'In':[5,5,5,5,8,11,14,14,14,63,63,63,63,128,15,15,15,15,15,15,15,15,143,143,143,10,10,10,10,10,13,15,15],
                 'load' :'000111000000110000000110000011100',
                 'inc'  :'011110001111111111111000011111100',
                 'reset':'000000000000000001000000000010000'
                },
                ['out']
            ),
            {'out':[0,0,1,2,5,8,11,11,11,12,13,14,15,63,128,129,130,131,0,1,2,3,15,143,143,143,144,145,146,0,10,13,13]}
        )

run_test(TestPC)

.
----------------------------------------------------------------------
Ran 1 test in 0.870s

OK
